# Twitter US Airline Sentiment Analysis

Given Twitter US Airline Sentiment Dataset, which contains data for over 14000 tweets, your task is to predict the sentiment of the tweet i.e. positive, negative or neutral.


Procedure:

    1.remove the stopwords, funcuations to get the clean data and lemmatization
    2.get the document containing whole words and find the most common N words
    3.see if training dataset have these common words and train the model
    4.test on testing dataset

In [103]:
import numpy as np
import pandas as pd
train_data=open('twitter_x_y_train.csv')
train_data=pd.read_csv(train_data)
test_data=open('twitter_x_test.csv')
test_data=pd.read_csv(test_data)

In [104]:
train_data.head()

,tweet_id,airline_sentiment,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,5.679000e+17,negative,Southwest,ColeyGirouard,0,"@SouthwestAir I am scheduled for the morning, ...",2015-02-17 20:16:29 -0800,Washington D.C.,Atlantic Time (Canada)
1,5.699890e+17,positive,Southwest,WalterFaddoul,0,@SouthwestAir seeing your workers time in and ...,2015-02-23 14:36:22 -0800,"Indianapolis, Indiana; USA",Central Time (US & Canada)
2,5.680890e+17,positive,United,LocalKyle,0,@united Flew ORD to Miami and back and had gr...,2015-02-18 08:46:29 -0800,Illinois,Central Time (US & Canada)
3,5.689280e+17,negative,Southwest,amccarthy19,0,@SouthwestAir @dultch97 that's horse radish 😤🐴,2015-02-20 16:20:26 -0800,NaN,Atlantic Time (Canada)
4,5.685940e+17,negative,United,J_Okayy,0,@united so our flight into ORD was delayed bec...,2015-02-19 18:13:11 -0800,NaN,Eastern Time (US & Canada)


In [105]:
x_train=train_data['text']
x_train.head()

0    @SouthwestAir I am scheduled for the morning, ...
1    @SouthwestAir seeing your workers time in and ...
2    @united Flew ORD to Miami and back and  had gr...
3       @SouthwestAir @dultch97 that's horse radish 😤🐴
4    @united so our flight into ORD was delayed bec...
Name: text, dtype: object

In [106]:
y_train=train_data['airline_sentiment']
y_train.head()

0    negative
1    positive
2    positive
3    negative
4    negative
Name: airline_sentiment, dtype: object

In [107]:
test_data.head()

,tweet_id,airline,name,retweet_count,text,tweet_location,user_timezone
0,5.696820e+17,American,zsalim03,0,@AmericanAir In car gng to DFW. Pulled over 1h...,Texas,Central Time (US & Canada)
1,5.696080e+17,American,sa_craig,0,"@AmericanAir after all, the plane didn’t land ...","College Station, TX",Central Time (US & Canada)
2,5.678790e+17,Southwest,DanaChristos,1,@SouthwestAir can't believe how many paying cu...,CT,Eastern Time (US & Canada)
3,5.697580e+17,US Airways,rossj987,0,@USAirways I can legitimately say that I would...,"Washington, D.C.",Eastern Time (US & Canada)
4,5.699010e+17,American,tranpham18,0,@AmericanAir still no response from AA. great ...,New York City,Eastern Time (US & Canada)


In [108]:
x_test=test_data['text']
x_test.head()

0    @AmericanAir In car gng to DFW. Pulled over 1h...
1    @AmericanAir after all, the plane didn’t land ...
2    @SouthwestAir can't believe how many paying cu...
3    @USAirways I can legitimately say that I would...
4    @AmericanAir still no response from AA. great ...
Name: text, dtype: object

#### 1. Get clean data-->remove stop words and functuations

In [109]:
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import string 
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /Users/fy/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [110]:
stop=stopwords.words('english')
punctuations=list(string.punctuation)
stop=stop+punctuations
stop

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [111]:
x_train=np.array(x_train)
y_train=np.array(y_train)
x_test=np.array(x_test)

**dealing with train data**

In [112]:
x_trainfinal=[]
for i in range(len(x_train)):
    a=word_tokenize(x_train[i])
    clean_words=[w for w in a if not w in stop]
    x_trainfinal.append(clean_words)

In [113]:
documents=[]
for i in range(len(x_trainfinal)):
    documents.append((x_trainfinal[i],y_train[i]))
documents[:5]

[(['SouthwestAir',
   'I',
   'scheduled',
   'morning',
   '2',
   'days',
   'fact',
   'yes..not',
   'sure',
   'evening',
   'flight',
   'one',
   'Cancelled',
   'Flightled'],
  'negative'),
 (['SouthwestAir',
   'seeing',
   'workers',
   'time',
   'time',
   'going',
   'beyond',
   'I',
   'love',
   'flying',
   'guys',
   'Thank'],
  'positive'),
 (['united',
   'Flew',
   'ORD',
   'Miami',
   'back',
   'great',
   'crew',
   'service',
   'legs',
   'THANKS'],
  'positive'),
 (['SouthwestAir', 'dultch97', "'s", 'horse', 'radish', '😤🐴'], 'negative'),
 (['united',
   'flight',
   'ORD',
   'delayed',
   'Air',
   'Force',
   'One',
   'last',
   'flight',
   'SBN',
   '8:20',
   '5',
   'mins',
   'landed'],
  'negative')]

In [114]:
import random
random.shuffle(documents)#randomly select
documents[:5]

[(['united', 'chase', 'says', 'referral', 'bonus', 'Thanks'], 'neutral'),
 (['Bruh', '“', 'VirginAmerica', 'giannilee', 'Turn', 'VXSafetyDance', '”'],
  'neutral'),
 (['SouthwestAir',
   'SW',
   'rocks',
   'thanks',
   'reply',
   'follow',
   'Rebooked',
   'earlier',
   'flight'],
  'positive'),
 (['SouthwestAir', 'area', 'pic', 'taken', "n't", 'see', 'albums'],
  'negative'),
 (['united', 'going', 'Nothing', 'like', 'always'], 'negative')]

#### 2. Lemmatization

In [115]:
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
lemmatizer=WordNetLemmatizer()

In [116]:
from nltk.corpus import wordnet
def get_simple_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else: 
        return wordnet.NOUN

In [117]:
def clean_review(words):
    output_words=[]
    for w in words:
        if w.lower() not in stop:
            pos=pos_tag([w])
            clean_word=lemmatizer.lemmatize(w,pos=get_simple_pos(pos[0][1]))
            output_words.append(clean_word.lower())
    return output_words    

In [118]:
document=[(clean_review(document),category) for document,category in documents]
document[0]

(['united', 'chase', 'say', 'referral', 'bonus', 'thanks'], 'neutral')

**dealing with test data**

In [159]:
x_testfinal=[]
for i in range(len(x_test)):
    a=word_tokenize(x_test[i])
    clean_words=[w for w in a if not w in stop]
    x_testfinal.append(clean_words)

In [161]:
documentst=[]
for i in range(len(x_testfinal)):
    documentst.append(x_testfinal[i])
documentst[:1]

[['AmericanAir',
  'In',
  'car',
  'gng',
  'DFW',
  'Pulled',
  '1hr',
  'ago',
  'icy',
  'roads',
  'On-hold',
  'AA',
  'since',
  '1hr',
  'Ca',
  "n't",
  'reach',
  'arpt',
  'AA2450',
  'Wat',
  '2']]

In [162]:
import random
random.shuffle(documentst)#randomly select
documentst[:1]

[['AmericanAir', 'I', "'m", 'great', 'thanks', 'keep', 'good', 'work']]

In [167]:
documentt=[clean_review(document)for document in documentst]
documentt[0]

['americanair', "'m", 'great', 'thanks', 'keep', 'good', 'work']

# Method 1

#### 3. Build features set

In [82]:
all_words=[]
for doc in document:
    all_words+=doc[0]

In [177]:
len(all_words)

119935

In [85]:
freq=nltk.FreqDist(all_words)
common=freq.most_common(2000)
feature=[i[0] for i in common]
feature[:10]

['flight',
 'united',
 'usairways',
 'americanair',
 'southwestair',
 'jetblue',
 'get',
 "n't",
 "'s",
 'http']

In [86]:
def get_feature_dict(words):
    current_feature={}
    words_set=set(words)
    for w in feature:
        current_feature[w]=w in words_set
    return  current_feature

In [87]:
training_data=[(get_feature_dict(doc),category) for doc,category in document]

In [216]:
testing_data=[get_feature_dict(doc) for doc in documentt]

In [145]:
training_data[0]

({'flight': True,
  'united': False,
  'usairways': True,
  'americanair': False,
  'southwestair': False,
  'jetblue': False,
  'get': False,
  "n't": True,
  "'s": False,
  'http': False,
  'hour': False,
  'thanks': False,
  'cancel': False,
  '...': False,
  'u': False,
  'service': False,
  'help': False,
  'customer': False,
  'time': False,
  'call': True,
  'bag': False,
  'wait': False,
  'go': False,
  'plane': False,
  'make': True,
  'fly': False,
  'need': True,
  'amp': False,
  'hold': False,
  "'m": False,
  '2': False,
  'thank': False,
  'would': False,
  'still': False,
  'one': False,
  'try': False,
  'airline': False,
  'gate': False,
  'day': False,
  'please': False,
  'back': False,
  'ca': False,
  'take': False,
  'delayed': False,
  'book': False,
  'say': False,
  'virginamerica': False,
  'flightled': False,
  'bad': False,
  'seat': False,
  'delay': False,
  "'ve": False,
  "''": False,
  '``': False,
  'like': False,
  'change': False,
  'phone': False,

#### 5. Fit the model

In [207]:
from nltk import NaiveBayesClassifier 

In [208]:
cl=NaiveBayesClassifier.train(training_data)
accuracy=nltk.classify.accuracy(cl,training_data)
accuracy

0.8269581056466302

In [213]:
training_data111=[get_feature_dict(doc) for doc,category in document]
training_data111[1].keys()

dict_keys(['flight', 'united', 'usairways', 'americanair', 'southwestair', 'jetblue', 'get', "n't", "'s", 'http', 'hour', 'thanks', 'cancel', '...', 'u', 'service', 'help', 'customer', 'time', 'call', 'bag', 'wait', 'go', 'plane', 'make', 'fly', 'need', 'amp', 'hold', "'m", '2', 'thank', 'would', 'still', 'one', 'try', 'airline', 'gate', 'day', 'please', 'back', 'ca', 'take', 'delayed', 'book', 'say', 'virginamerica', 'flightled', 'bad', 'seat', 'delay', "'ve", "''", '``', 'like', 'change', 'phone', 'agent', 'miss', 'late', 'today', 'guy', 'know', 'ticket', 'work', 'airport', 'could', 'give', 'minute', 'way', 'great', 'lose', 'well', 'check', '3', 'travel', 'never', 'even', 'min', 'problem', 'see', 'want', 'really', 'hr', 'home', "'re", 'look', 'tomorrow', 'weather', '4', 'dm', 'last', 'good', 'another', 'use', 'people', 'love', 'email', 'issue', 'sit', 'new', 'let', 'told', 'first', "'ll", 'luggage', 'much', 'yes', 'ever', 'number', 'due', 'someone', 'next', 'trip', 'baggage', 'reserv

In [214]:
cl.classify_many(training_data111)

['positive',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'positive',


In [218]:
a=cl.classify_many(testing_data)

In [222]:
a

['positive',
 'neutral',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'negative',
 'negative',
 'neutral',
 'positive',
 'positive',
 'negative',
 'negative',
 'neutral',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'negative',
 'neutral',
 'negative',
 'neutral',
 'positive',
 'neutral',
 'neutral',
 'neutral',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'positive',
 'positive',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'positive',
 'neutral',
 'negative',
 'negative',
 'negative',
 'neutral',
 'positive',
 'negative',
 'negative',
 'positive',
 'ne

In [223]:
np.savetxt("predic1.csv", a, delimiter=",", fmt='%s')

In [92]:
cl.show_most_informative_features(15)

Most Informative Features
                   kudos = True           positi : negati =     51.9 : 1.0
               fantastic = True           positi : negati =     34.2 : 1.0
                favorite = True           positi : negati =     34.2 : 1.0
             outstanding = True           positi : negati =     29.1 : 1.0
                  dragon = True           neutra : negati =     28.5 : 1.0
                 helpful = True           positi : neutra =     25.4 : 1.0
                   flyfi = True           positi : negati =     24.1 : 1.0
                   thank = True           positi : negati =     23.0 : 1.0
                 awesome = True           positi : negati =     22.6 : 1.0
                    hold = True           negati : positi =     22.3 : 1.0
               beautiful = True           positi : negati =     21.5 : 1.0
                      hr = True           negati : positi =     21.4 : 1.0
                  street = True           neutra : negati =     20.6 : 1.0

# Method 2

In [94]:
from sklearn.feature_extraction.text import CountVectorizer

In [97]:
text_documents = [" ".join(doc) for doc, category in document]
text_documents

["usairways point n't need sorry need revers agt pick call maybe make 11:30 flight today",
 'united orthodoc on-call in-flight glad service',
 'usairways bougth ticket fligths twice dont refund money big problem company refund money dont u pay back',
 'jetblue airway receives new coverage analyst bank america jblu legacy http //t.co/doruxbqla1',
 'southwestair well need something aim',
 'united doumented via link however four month response suggest shall expect one',
 "united please see flight attendant n't enough room head bag carry save time",
 'southwestair realize wrong departure date thanks make change easy',
 'united nyc flight delayed miss connection sacramento chicago',
 'usairways travel 2 year old would appreciate hold end good business host u unanticipated layover',
 'americanair get bit ridiculous make miss 2 full day work pay cab back forth',
 "americanair `` airport snow removal method 22 ... '' keep good work folk cessna 's become 747 's http //t.co/oumc1lrxdn",
 'united

In [169]:
text_documentst = [" ".join(doc) for doc in documentt]
text_documentst

["americanair 'm great thanks keep good work",
 'united follow please airline beautifull ♥',
 "jetblue drwinston001 think 'll need selfie proof ...",
 "united love hate relationship day 're good others terribly awful sadden",
 "jetblue apparently substitute pilot flight 69 bos fll ca n't found info avail counter",
 "americanair try talk someone almost 6 hr cancelled flightled flight wo n't pick phone pathetic amp sad",
 'southwestair flight leave dc today flight cancelled flightled hold hour',
 'southwestair past 2 yr earn a-list status thru promo-fly 3 rnd trip 2-3 month get option loyal',
 'usairways two woman workng phl bag check term c 11:30 et fire rude pushy treat customer disrespect',
 'usairways hundred people line less half desk man clt help',
 "southwestair best credit card use/open get mile y'all",
 "usairways n't hire people deal call volume absolute garbage incompetent",
 'usairways chairman phone number use',
 'jetblue try charge u 550 dollar cash ticket paid month advanc

In [186]:
count_vec = CountVectorizer(max_features = 2000,ngram=(1,2))
x_train_features = count_vec.fit_transform(text_documents)
x_train_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [187]:
x_test_features = count_vec.transform(text_documentst)
x_test_features.todense()

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [188]:
from sklearn.svm import SVC
svc = SVC()
svc.fit(x_train_features, y_train)

//anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='rbf', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [189]:
svc.score(x_train_features, y_train)

0.6239526411657559

In [126]:
svc.predict(x_train_features)

array(['negative', 'negative', 'negative', ..., 'negative', 'negative',
       'negative'], dtype=object)

In [190]:
prediction=svc.predict(x_test_features)

In [191]:
predic=[]
for i in prediction:
    predic.append(i)
predic

['negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',
 'negative',

In [193]:
np.savetxt("predic.csv", predic, delimiter=",", fmt='%s')